## Import all required libraries and functions 

In [1]:
# Import necessary libraries
import os
import time
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Lambda, Flatten, GlobalMaxPooling2D
from tensorflow.keras.applications import VGG16, ResNet50, InceptionV3

import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn, retinanet
from torchvision.transforms import functional as F

<b> saving the Start Time

In [2]:
main_start_time=time.time()

## Code for CNN
<b> 1. Setup and Preprocessing

In [3]:

# Define the root folder
root_folder = '/Users/ravkothu/Documents/Personal_items_at_Oracle/Master_Degree/University_of_San_Diego/Online_Masters/MS_in_Applied_AI/Subjects_and_Resources/AAI-501_Introduction_to_AI/AAI-501_Final_Team_Project/pneumonia_detection/dataset/chest_xray_nrm_padded'

# Get the subfolders in the root folder
subfolders = [f for f in os.listdir(root_folder) if os.path.isdir(os.path.join(root_folder, f))]

# Assign subfolders to train_folder and test_folder based on their names
for folder in subfolders:
    if 'train' in folder.lower():
        train_folder = os.path.join(root_folder, folder)
        print(f"Found Train Folder --->\t {train_folder}")
    elif 'test' in folder.lower():
        test_folder = os.path.join(root_folder, folder)
        print(f"Found Test Folder --->\t {test_folder}")

# Define class labels dynamically from folder names
class_labels = {}
for element in subfolders:
    element_path = os.path.join(root_folder, element)  # Path identified
    for idx, folder in enumerate(os.listdir(element_path)):
        key=f"{element}_{folder}"
        if 'normal' in folder.lower():
            class_labels[key] = 0
        else:
            class_labels[key] = 1
            
print("\n")
class_labels

Found Train Folder --->	 /Users/ravkothu/Documents/Personal_items_at_Oracle/Master_Degree/University_of_San_Diego/Online_Masters/MS_in_Applied_AI/Subjects_and_Resources/AAI-501_Introduction_to_AI/AAI-501_Final_Team_Project/pneumonia_detection/dataset/chest_xray_nrm_padded/train_nrm_padded
Found Test Folder --->	 /Users/ravkothu/Documents/Personal_items_at_Oracle/Master_Degree/University_of_San_Diego/Online_Masters/MS_in_Applied_AI/Subjects_and_Resources/AAI-501_Introduction_to_AI/AAI-501_Final_Team_Project/pneumonia_detection/dataset/chest_xray_nrm_padded/test_nrm_padded




{'train_nrm_padded_PNEUMONIA_nrm_padded': 1,
 'train_nrm_padded_NORMAL_nrm_padded': 0,
 'test_nrm_padded_PNEUMONIA_nrm_padded': 1,
 'test_nrm_padded_NORMAL_nrm_padded': 0}

## CNN with Adaptive Pooling

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Lambda, Flatten
from tensorflow.keras.models import Model

# CNN with Global Average Pooling (similar to adaptive pooling)
def build_cnn_with_adaptive_pooling(input_shape):
    inputs = Input(shape=input_shape)
    x = Conv2D(32, (3, 3), activation='relu')(inputs)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = GlobalAveragePooling2D()(x)  # Global Average Pooling instead of Adaptive Pooling
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    outputs = Dense(2, activation='softmax')(x)  # Assuming binary classification
    model = Model(inputs, outputs)
    return model

# Example usage
if __name__ == "__main__":
    input_shape = (224, 224, 3)  # Example input shape
    cnn_model = build_cnn_with_adaptive_pooling(input_shape)
    cnn_model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 110,018 (429.76 KB)

 Trainable params: 110,018 (429.76 KB)

 Non-trainable params: 0 (0.00 B)

### Load and Preprocess Images

In [5]:
# # root_folder = 'C:/Users/Soumiz/Downloads/AII 501 Project Pneumonia/ChestXRay2017_nrm'

# Define ImageDataGenerator for training and testing
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Create the data generators
train_generator = train_datagen.flow_from_directory(
    os.path.join(root_folder, 'train_nrm_padded'),
    target_size=(224, 224),
    batch_size=8,                                   # Batch size can be changed to bigger 
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    os.path.join(root_folder, 'test_nrm_padded'),
    target_size=(224, 224),
    batch_size=8,                                   # Batch size can be changed to bigger 
    class_mode='categorical')


Found 5232 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


### Get Pre-trained Model

In [6]:
input_shape = (224, 224, 3)
model_name = 'CNN'   
model = cnn_model


### Compile and Train the Model

In [7]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

start_time=time.time()
# Train the model
print("start Time=",time.time()-start_time)

model.fit(train_generator, epochs=50, validation_data=test_generator)   # change epoch to higher value for proper evaluation

print("Time taken to Compile and Train=",time.time()-start_time)

start Time= 3.409385681152344e-05
Epoch 1/50


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


654/654 ━━━━━━━━━━━━━━━━━━━━ 238s 360ms/step - accuracy: 0.7815 - loss: 0.5324 - val_accuracy: 0.8205 - val_loss: 0.4155
Epoch 2/50
654/654 ━━━━━━━━━━━━━━━━━━━━ 225s 341ms/step - accuracy: 0.8382 - loss: 0.3382 - val_accuracy: 0.8125 - val_loss: 0.4300
Epoch 3/50
654/654 ━━━━━━━━━━━━━━━━━━━━ 226s 343ms/step - accuracy: 0.8276 - loss: 0.3508 - val_accuracy: 0.8606 - val_loss: 0.3496
Epoch 4/50
654/654 ━━━━━━━━━━━━━━━━━━━━ 223s 338ms/step - accuracy: 0.8288 - loss: 0.3504 - val_accuracy: 0.8494 - val_loss: 0.3718
Epoch 5/50
654/654 ━━━━━━━━━━━━━━━━━━━━ 224s 339ms/step - accuracy: 0.8247 - loss: 0.3516 - val_accuracy: 0.8574 - val_loss: 0.3704
Epoch 6/50
654/654 ━━━━━━━━━━━━━━━━━━━━ 229s 346ms/step - accuracy: 0.8455 - loss: 0.3266 - val_accuracy: 0.8606 - val_loss: 0.3779
Epoch 7/50
654/654 ━━━━━━━━━━━━━━━━━━━━ 229s 347ms/step - accuracy: 0.8482 - loss: 0.3197 - val_accuracy: 0.8606 - val_loss: 0.3477
Epoch 8/50
654/654 ━━━━━━━━━━━━━━━━━━━━ 226s 343ms/step - accuracy: 0.8532 - loss: 0.31

654/654 ━━━━━━━━━━━━━━━━━━━━ 229s 347ms/step - accuracy: 0.9424 - loss: 0.1531 - val_accuracy: 0.8125 - val_loss: 0.4860
Time taken to Compile and Train= 11255.480288028717


### Evaluate the Model

In [8]:
# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")

78/78 ━━━━━━━━━━━━━━━━━━━━ 23s 296ms/step - accuracy: 0.8072 - loss: 0.5106
Test accuracy: 0.8125


### Evaluation time

In [9]:
Total_time=time.time()-main_start_time
print("Total Execution time=",Total_time)

Total Execution time= 11329.838241100311


## Pre-trained Models [VGG16] with Global Pooling

import tensorflow as tf
from tensorflow.keras.applications import VGG16, ResNet50, InceptionV3
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Pre-trained Models with Global Pooling
def get_pretrained_model(model_name, input_shape):
    if model_name == 'vgg16':
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
        x = base_model.output
        x = GlobalAveragePooling2D()(x)  # Global Average Pooling
    elif model_name == 'resnet50':
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
        x = base_model.output
        x = GlobalAveragePooling2D()(x)  # Global Average Pooling
    elif model_name == 'inceptionv3':
        base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)
        x = base_model.output
        x = GlobalAveragePooling2D()(x)  # Global Average Pooling
    else:
        raise ValueError('Unsupported model name.')
    x = Dense(128, activation='relu')(x)
    outputs = Dense(2, activation='softmax')(x)  # Assuming binary classification
    model = Model(base_model.input, outputs)
    return model

# Example usage
if __name__ == "__main__":
    input_shape = (224, 224, 3)  # Example input shape
    model_name = 'vgg16'  # Change to 'resnet50' or 'inceptionv3' as needed
    pretrained_model = get_pretrained_model(model_name, input_shape)
    pretrained_model.summary()


## Object Detection Models

import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn, retinanet_resnet50_fpn

# Object Detection Models
def get_object_detection_model(model_name):
    if model_name == 'fasterrcnn':
        model = fasterrcnn_resnet50_fpn(weights=True)
    elif model_name == 'retinanet':
        model = retinanet_resnet50_fpn(weights=True)
    else:
        raise ValueError('Unsupported object detection model.')
    return model

# Example usage
if __name__ == "__main__":
    detection_model_name = 'fasterrcnn'  # Change to 'retinanet' as needed
    detection_model = get_object_detection_model(detection_model_name)
    print(f"Object detection model ({detection_model_name}) loaded.")
